In [31]:
# ==========================================
# Store Sales Excel Consolidation (Value Mode)
# ==========================================
# - 各店舗のExcelを読み込み、指定シートから数値データを抽出
# - 出力は ./output/consolidated_sales.xlsx
# ------------------------------------------
# Author : kokoyugura
# Date   : 2025-11
# Env    : Python 3.10+, openpyxl, pandas
# ==========================================


In [32]:
# === Library Import ===
import time
from pathlib import Path
import pandas as pd
from openpyxl import load_workbook

In [33]:
# === Configuration ===
ROOT = Path.cwd()                 # スクリプト格納先
DATA_PATH = ROOT / "data"         # 店舗Excel格納先
TARGET_SHEET = "October"          # 対象シート名

files = list(DATA_PATH.glob("*.xlsx")) + list(DATA_PATH.glob("*.xlsm"))  # 両対応
file_names = [f.name for f in files]      # 表示用（ファイル名）
file_paths = [str(f) for f in files]      # 処理用（フルパス文字列）

#以下、公開用のためコメントアウト
# print("実行ディレクトリ：", ROOT)  
# print("参照フォルダ　　：", DATA_PATH)
# print("参照シート　　　：", TARGET_SHEET)
# print("検出ファイル数　：", len(file_names))
# print("抽出ファイル一覧：", file_names)

print("実行ディレクトリ： [Hidden for public]")
print("参照フォルダ　　： ./data")
print("参照シート　　　：", TARGET_SHEET)
print("検出ファイル数　：", len(file_names))
print("抽出ファイル一覧：", file_names)

実行ディレクトリ： [Hidden for public]
参照フォルダ　　： ./data
参照シート　　　： October
検出ファイル数　： 1
抽出ファイル一覧： ['銀座店.xlsx']


In [34]:
# === Core Extraction Function ===
def extract_store_data(filepath: str, sheet_name: str):
    """
    店舗Excelから数値データを抽出
    """
    records = []
    wb = load_workbook(filepath, data_only=True)  # 数値のみ取得
    if sheet_name not in wb.sheetnames:
        raise FileNotFoundError(f"シート '{sheet_name}' が見つかりません。")
    ws = wb[sheet_name]

    # 固定セル
    place_code = ws["C3"].value
    place_name = ws["C4"].value
    budget     = ws["C5"].value

    # 明細行（行・列位置は構造に応じて変更）
    for row in range(4, 14):
        plan_no     = ws[f"E{row}"].value
        plan_name   = ws[f"F{row}"].value
        plan_class1 = ws[f"G{row}"].value
        plan_class2 = ws[f"H{row}"].value
        plan_budget = ws[f"I{row}"].value
        plan_start  = ws[f"J{row}"].value
        plan_end    = ws[f"K{row}"].value

        records.append([
            place_code, place_name, budget, 
            plan_no, plan_name, plan_class1, plan_class2,
            plan_budget, plan_start, plan_end
        ])
    wb.close()
    return records

In [35]:
# === Consolidate Process ===
def consolidate_all_stores(file_list):
    """
    全店舗のExcelを読み込み、DataFrame用リストに統合
    """
    start = time.time()
    data = []

    for path in file_list:
        store_name = Path(path).stem
        print(f"処理中: {store_name}")
        try:
            records = extract_store_data(path, TARGET_SHEET)
            data.extend(records)
        except Exception as e:
            print(f"⚠️ エラー: {store_name} - {e}")

    elapsed = time.time() - start
    print("\n✅ 全店舗処理完了")
    print(f"総レコード数: {len(data)}")
    print(f"処理時間: {elapsed:.2f} 秒（約 {elapsed/60:.1f} 分）")
    return data

In [36]:
# === 実行 ===
data = consolidate_all_stores(file_paths)

# === DataFrame & Export ===
columns = [
    "Code", "Name", "Budget",
    "PlanNo", "PlanName", "Class1", "Class2",
    "PlanBudget", "Start", "End"
]
df = pd.DataFrame(data, columns=columns)

OUTPUT_DIR = ROOT / "output"
OUTPUT_DIR.mkdir(exist_ok=True)
out_path = OUTPUT_DIR / "consolidated_sales.xlsx"
df.to_excel(out_path, index=False)

print("📦 出力完了： ./output/consolidated_sales.xlsx")

処理中: 銀座店

✅ 全店舗処理完了
総レコード数: 10
処理時間: 0.01 秒（約 0.0 分）
📦 出力完了： ./output/consolidated_sales.xlsx
